In [1]:
from mc_openapi.doml_mc.intermediate_model.metamodel import parse_metamodel, parse_inverse_associations
import yaml
with open("../assets/doml_meta.yaml") as mmf:
    mmdoc = yaml.load(mmf, yaml.Loader)
mm = parse_metamodel(mmdoc)
inv_assoc = parse_inverse_associations(mmdoc)

In [2]:
import prettyprinter as pp
from prettyprinter.prettyprinter import IMPLICIT_MODULES
pp.install_extras(include=['dataclasses'])

In [3]:
IMPLICIT_MODULES.add('doml_mc.intermediate_model.metamodel')
pp.pprint(mm["application_SoftwareComponent"])

mc_openapi.doml_mc.intermediate_model.metamodel.DOMLClass(
    name='application_SoftwareComponent',
    superclass='application_ApplicationComponent',
    attributes={
        'isPersistent': mc_openapi.doml_mc.intermediate_model.metamodel.DOMLAttribute(
            name='isPersistent',
            type='Boolean',
            multiplicity=('1', '1'),
            default=False
        )
    },
    associations={
        'exposedInterfaces': mc_openapi.doml_mc.intermediate_model.metamodel.DOMLAssociation(
            name='exposedInterfaces',
            class_='application_SoftwareInterface',
            multiplicity=('0', '*')
        ),
        'consumedInterfaces': mc_openapi.doml_mc.intermediate_model.metamodel.DOMLAssociation(
            name='consumedInterfaces',
            class_='application_SoftwareInterface',
            multiplicity=('0', '*')
        )
    }
)


### Load the DOML document here:

In [4]:
from mc_openapi.doml_mc.xmi_parser.doml_model import parse_doml_model
# doml_document_path = "../../tests/doml/nginx-openstack_v2.domlx"
doml_document_path = "../../tests/doml/nginx-openstack_v2_wrong.domlx"
with open(doml_document_path, "rb") as xmif:
    doc = xmif.read()

doml_model = parse_doml_model(doc, mm)
print(doml_model)

In [6]:
from mc_openapi.doml_mc.intermediate_model.doml_model2im import doml_model_to_im
from mc_openapi.doml_mc.intermediate_model.doml_element import reciprocate_inverse_associations
im = doml_model_to_im(doml_model, mm)
reciprocate_inverse_associations(im, inv_assoc)

In [7]:
IMPLICIT_MODULES.add('doml_mc.intermediate_model.doml_element')
pp.pprint(im)

{
    'nginx': mc_openapi.doml_mc.intermediate_model.doml_element.DOMLElement(
        name='nginx',
        class_='application_SoftwareComponent',
        attributes={'commons_DOMLElement::name': 'nginx'},
        associations={
            'application_SoftwarePackage::consumedInterfaces': set(),
            'application_SoftwarePackage::exposedInterfaces': set()
        }
    ),
    'vm1': mc_openapi.doml_mc.intermediate_model.doml_element.DOMLElement(
        name='vm1',
        class_='infrastructure_VirtualMachine',
        attributes={'commons_DOMLElement::name': 'vm1'},
        associations={'infrastructure_ComputingNode::ifaces': set()}
    ),
    'net1': mc_openapi.doml_mc.intermediate_model.doml_element.DOMLElement(
        name='net1',
        class_='infrastructure_Network',
        attributes={
            'commons_DOMLElement::name': 'net1',
            'infrastructure_Network::address_lb': 268435456,
            'infrastructure_Network::address_ub': 268435711
        }

In [8]:
unbound_elems_n = 1
unbound_elems = [f"unbound{i}" for i in range(unbound_elems_n)]

In [9]:
from z3 import Solver

from mc_openapi.doml_mc.z3.metamodel_encoding import (
    def_association_rel_and_assert_constraints,
    def_attribute_rel_and_assert_constraints,
    mk_association_sort_dict,
    mk_attribute_sort_dict,
    mk_class_sort_dict
)
from mc_openapi.doml_mc.z3.im_encoding import (
    assert_im_associations_q,
    assert_im_attributes,
    def_elem_class_f_and_assert_classes,
    mk_elem_sort_dict,
    mk_stringsym_sort_dict
)
from mc_openapi.doml_mc.z3.utils import mk_adata_sort

solver = Solver()

class_sort, class_ = mk_class_sort_dict(mm)
assoc_sort, assoc = mk_association_sort_dict(mm)
attr_sort, attr = mk_attribute_sort_dict(mm)
elem_sort, elem = mk_elem_sort_dict(im, unbound_elems)
ss_sort, ss = mk_stringsym_sort_dict(im, mm)
AData = mk_adata_sort(ss_sort)
elem_class_f = def_elem_class_f_and_assert_classes(
    im,
    solver,
    elem_sort,
    elem,
    class_sort,
    class_
)
attr_rel = def_attribute_rel_and_assert_constraints(
    mm,
    solver,
    attr_sort,
    attr,
    class_,
    elem_class_f,
    elem_sort,
    AData,
    ss
)
assert_im_attributes(
    attr_rel,
    solver,
    im,
    mm,
    elem,
    attr_sort,
    attr,
    AData,
    ss
)
assoc_rel = def_association_rel_and_assert_constraints(
    mm,
    solver,
    assoc_sort,
    assoc,
    class_,
    elem_class_f,
    elem_sort,
    inv_assoc
)
assert_im_associations_q(
    assoc_rel,
    solver,
    {k: v for k, v in im.items() if k not in unbound_elems},
    elem,
    assoc_sort,
    assoc,
)

In [10]:
# solver.push()

### Metamodel statistics

In [11]:
from mc_openapi.doml_mc.intermediate_model.metamodel import get_subclasses_dict

print("Number of classes:", len(mm))
print(
    "Number of tuples in the subclass relation:",
    sum(len(s) for s in get_subclasses_dict(mm).values())
)
n_attrs = sum(len(c.attributes) for c in mm.values())
print("Number of attributes:", n_attrs)
print("Number of attribute assertions:", 3 * n_attrs)
n_assocs = sum(len(c.associations) for c in mm.values())
print("Number of associations:", n_assocs)
print("Number of associations assertions:", 3 * n_assocs + len(inv_assoc))



Number of classes: 36
Number of tuples in the subclass relation: 91
Number of attributes: 24
Number of attribute assertions: 72
Number of associations: 47
Number of associations assertions: 146


### DOML model statistics

In [12]:
n_elems = len(im)
print("Number of elements:", n_elems)
n_attrs_im = sum(len(e.attributes) for e in im.values())
print("Number of attributes (IM):", n_attrs_im)
n_assocs_im = sum(len(a) for e in im.values() for a in e.associations.values())
print("Number of associations (IM):", n_assocs_im)
print("Number of string symbols:", len(ss))
print("Number of IM attribute assertions:", n_elems)
print("Number of IM association assertions:", n_elems ** 2)

Number of elements: 8
Number of attributes (IM): 10
Number of associations (IM): 7
Number of string symbols: 10
Number of IM attribute assertions: 8
Number of IM association assertions: 64


In [13]:
solver.check()

sat

In [14]:
solver.statistics()

(:added-eqs               4046
 :arith-make-feasible     5
 :arith-max-columns       6
 :binary-propagations     208
 :conflicts               26
 :datatype-accessor-ax    246
 :datatype-constructor-ax 1534
 :datatype-splits         249
 :decisions               7349
 :del-clause              14
 :final-checks            8
 :max-generation          4
 :max-memory              22.49
 :memory                  22.49
 :mk-bool-var             1342
 :mk-clause               473
 :num-allocs              2766758
 :num-checks              1
 :propagations            1442
 :quant-instantiations    339
 :restarts                3
 :rlimit-count            958188
 :time                    0.16)

In [15]:
from z3 import Consts, ForAll, Exists, Implies, And, Or

In [16]:
# All VMs have a network interface.
vm, iface = Consts(
    "vm iface", elem_sort
)
vmIfaceAssertion = ForAll(
    [vm],
    Implies(
        elem_class_f(vm) == class_["infrastructure_VirtualMachine"],
        Exists(
            [iface],
            And(
                assoc_rel(vm, assoc["infrastructure_ComputingNode::ifaces"], iface)
            )
        )
    )
)
solver.assert_and_track(vmIfaceAssertion, "vm_iface")

In [17]:
solver.check()

sat

In [18]:
solver.statistics().memory

23.16

In [19]:
m = solver.model()
print(m)

[attribute_st_types infrastructure_VirtualMachine::sizeDescription = True,
 attribute_mult_ub infrastructure_Network::address_ub = True,
 attribute_values vm1 = True,
 association_st_classes concrete_ConcreteInfrastructure::storages = True,
 association_st_classes concrete_VirtualMachine::maps = True,
 attribute_mult_ub infrastructure_Storage::cost = True,
 attribute_st_types commons_DOMLElement::name = True,
 attribute_st_types infrastructure_Network::address_lb = True,
 association_mult_ub concrete_FunctionAsAService::maps = True,
 associations con_infra con_infra = True,
 association_inverse infrastructure_Container::generatedFrom infrastructure_ContainerImage::generatedContainers = True,
 associations concrete_vm sg = True,
 association_mult_ub infrastructure_Container::generatedFrom = True,
 association_st_classes infrastructure_FunctionAsAService::ifaces = True,
 association_mult_ub infrastructure_VirtualMachine::location = True,
 association_st_classes concrete_Storage::maps = T

In [20]:
from itertools import product
for (e1n, e1), a, (e2n, e2) in product(elem.items(), assoc.values(), elem.items()):
    if (e1n in unbound_elems or e2n in unbound_elems) and m.eval(assoc_rel(e1, a, e2)):
        print(e1, a, e2)

vm1 infrastructure_ComputingNode::ifaces unbound0
net1 infrastructure_Network::connectedIfaces unbound0
unbound0 infrastructure_NetworkInterface::belongsTo net1
